In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id, col, expr, when, concat, lit, isnan
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.feature import VectorIndexer, VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator, BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.clustering import LDA, BisectingKMeans
import pyspark

In [2]:
!pip install TextBlob

In [3]:
spark

In [4]:
from textblob import TextBlob, Word

In [5]:
df.show(10)

+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
| id|id_news|               title|   publication|              author|      date|  year|month| url|             content|
+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
|  0|  17283|House Republicans...|New York Times|          Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON  —   C...|
|  1|  17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0|  6.0|null|After the bullet ...|
|  2|  17285|Tyrus Wong, ‘Bamb...|New York Times|        Margalit Fox|2017-01-06|2017.0|  1.0|null|When Walt Disney’...|
|  3|  17286|Among Deaths in 2...|New York Times|    William McDonald|2017-04-10|2017.0|  4.0|null|Death may be the ...|
|  4|  17287|Kim Jong-un Says ...|New York Times|       Choe Sang-Hun|2017-01-02|2017.0|  1.0|null|SEOUL, South Kore...|
|  5|  17288|Sick With a Cold,..

In [11]:
df.createOrReplaceTempView("news")

In [12]:
sqlDF = spark.sql("SELECT * FROM news where id = 0")
sqlDF.show(19)

+---+-------+--------------------+--------------+----------+----------+------+-----+----+--------------------+
| id|id_news|               title|   publication|    author|      date|  year|month| url|             content|
+---+-------+--------------------+--------------+----------+----------+------+-----+----+--------------------+
|  0|  17283|House Republicans...|New York Times|Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON  —   C...|
+---+-------+--------------------+--------------+----------+----------+------+-----+----+--------------------+



In [11]:
#df.write.parquet("csv_to_paraquet")
df_1 = spark.read.option("header","true").parquet("csv_to_paraquet")

In [8]:
df_1.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_news: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publication: string (nullable = true)
 |-- author: string (nullable = true)
 |-- date: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- url: string (nullable = true)
 |-- content: string (nullable = true)



In [13]:
df_1= df_1.fillna({'content': '', 'title': ''})

In [14]:
rz = df_1[['id','title','content']].rdd.map(lambda x: (x.id,x.title, TextBlob(x.content).sentiment)).collect()
rz

[('41272',
  'Dashcam video appears to show Fort Worth cop shooting man in the back',
  Sentiment(polarity=0.06725014116318465, subjectivity=0.29210191981931116)),
 ('41273',
  'Syria civil war: ceasefire underway',
  Sentiment(polarity=0.03549761552810333, subjectivity=0.3174290817583501)),
 ('41274',
  'Ex-cop’s retrial date set in Walter Scott killing',
  Sentiment(polarity=-0.03276515151515152, subjectivity=0.26179653679653686)),
 ('41275',
  'Israel risks sliding toward apartheid',
  Sentiment(polarity=0.05645247609533325, subjectivity=0.45804255268540955)),
 ('41276',
  'Does Melania Trump’s libel suit really threaten a free press?',
  Sentiment(polarity=0.06225605521166269, subjectivity=0.5153638521395528)),
 ('41277',
  'Conway wonders if Russian sanctions were intended to restrict Trump',
  Sentiment(polarity=0.09991976414390209, subjectivity=0.40591754988306705)),
 ('41278',
  'Nevada’s Heller says he’ll run for re-election to Senate in 2018',
  Sentiment(polarity=0.170977011

In [17]:
print(rz[:1000])

[('41272', 'Dashcam video appears to show Fort Worth cop shooting man in the back', Sentiment(polarity=0.06725014116318465, subjectivity=0.29210191981931116)), ('41273', 'Syria civil war: ceasefire underway', Sentiment(polarity=0.03549761552810333, subjectivity=0.3174290817583501)), ('41274', 'Ex-cop’s retrial date set in Walter Scott killing', Sentiment(polarity=-0.03276515151515152, subjectivity=0.26179653679653686)), ('41275', 'Israel risks sliding toward apartheid', Sentiment(polarity=0.05645247609533325, subjectivity=0.45804255268540955)), ('41276', 'Does Melania Trump’s libel suit really threaten a free press?', Sentiment(polarity=0.06225605521166269, subjectivity=0.5153638521395528)), ('41277', 'Conway wonders if Russian sanctions were intended to restrict Trump', Sentiment(polarity=0.09991976414390209, subjectivity=0.40591754988306705)), ('41278', 'Nevada’s Heller says he’ll run for re-election to Senate in 2018', Sentiment(polarity=0.17097701149425287, subjectivity=0.445689655